In [18]:
from qiskit_ibm_runtime import QiskitRuntimeService

token = '4c635c9b66b0ab2f37ce29d7bc33edc66f815847092ba236aa79748304b2cd4bbba6c35d8a4a6adc9b579b1990cfa8066f5220cbdc29044dc9bd59f1c1e1ea1b'
instance = 'ibm-q/open/main'

service = QiskitRuntimeService(
        channel='ibm_quantum',
        instance=instance,
        token=token
)

In [21]:
from qiskit import QuantumCircuit

def create_quantum_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits)
    # Apply Hadamard gate to each qubit
    for i in range(num_qubits):
        qc.h(i)
    # Apply CNOT gate between each pair of qubits
    for i in range(num_qubits - 1):
        qc.cx(i, i + 1)
    return qc

# Specify the number of qubits
num_qubits = 4  # Change this value as needed
qc = create_quantum_circuit(num_qubits)

print(qc)

     ┌───┐               
q_0: ┤ H ├──■────────────
     ├───┤┌─┴─┐          
q_1: ┤ H ├┤ X ├──■───────
     ├───┤└───┘┌─┴─┐     
q_2: ┤ H ├─────┤ X ├──■──
     ├───┤     └───┘┌─┴─┐
q_3: ┤ H ├──────────┤ X ├
     └───┘          └───┘


In [27]:
print(service.instances())
print(service.backends())

['ibm-q/open/main']
[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_sherbrooke')>, <IBMBackend('ibm_kyiv')>]


In [33]:
from qiskit_ibm_runtime import Session, Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import state_fidelity

# List of backends to iterate through
# Get the list of available backends
#backends = ['ibm_brisbane', 'ibm_sherbrooke', 'ibm_kyiv']
#backends =service.backends()
#print(backends)

backend_names = ['ibm_brisbane', 'ibm_sherbrooke', 'ibm_kyiv']

ideal_backend = service.backend(backend_names[0])
for backend_name in backend_names:
    backend = service.backend(backend_name)
    pass_Manager = generate_preset_pass_manager(optimization_level=1, backend=backend)

qc_transpiled = pass_Manager.run(qc)
operators_transpiled_list = [op.apply_layout(qc_transpiled.layout) for op in operators]

# Ideal state vector (assuming the ideal backend is the first one in the list)
ideal_backend = backends[0]
print(f"Ideal backend: {ideal_backend}")
with Session(backend=ideal_backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run(pubs=[qc])
    ideal_result = job.result()
    ideal_statevector = ideal_result.get_statevector()

# Store results
results = []

# Iterate through the backends and compare results
for backend_name in backends:
    with Session(service=service, backend=backend_name) as session:
        sampler = Sampler(mode=session)
        job = sampler.run(pubs=qc)
        result = job.result()
        prepared_statevector = result.get_statevector()
        
        # Calculate fidelity
        fidelity = state_fidelity(ideal_statevector, prepared_statevector)
        results.append((backend_name, fidelity))

# Display results
import pandas as pd

df = pd.DataFrame(results, columns=['Backend', 'Fidelity'])
print(df)

[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_sherbrooke')>, <IBMBackend('ibm_kyiv')>]
Ideal backend: <IBMBackend('ibm_brisbane')>


/home/jbedell/anaconda3/envs/pyenv-eigen/lib/python3.13/site-packages/qiskit_ibm_runtime/utils/validations.py:45: UserWarning: The 0-th circuit has no output classical registers so the result will be empty. Did you mean to add measurement instructions?
  warnings.warn(


IBMInputValueError: 'The instruction h on qubits (0,) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (https://docs.quantum.ibm.com/guides/transpile) for instructions to transform circuits and the primitive examples (https://docs.quantum.ibm.com/guides/primitives-examples) to see this coupled with operator transformations.'

In [ ]:
import matplotlib.pyplot as plt

# Display the results as a table
print(df)

# Plot the results
plt.figure(figsize=(10, 6))
plt.bar(df['Backend'], df['Fidelity'], color='blue')
plt.xlabel('Backend')
plt.ylabel('Fidelity')
plt.title('Fidelity of Quantum Circuit Execution on Different Backends')
plt.show()